In [2]:
import sqlite3
import pandas as pd

In [2]:
# List of CSV files
csv_files = [
    "Trials/parsed_data/darkness_bins.csv",
    "Trials/parsed_data/localization.csv",
    "Trials/parsed_data/luminecance_annotations.csv",
    "Trials/parsed_data/protein_sequences.csv"
]

# SQLite database file
database_file = "Trials/parsed_data/my_database.db"

# Create SQLite database and cursor
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

# Function to create a table in the database
def create_table(table_name, df):
    df.to_sql(table_name, conn, index=False, if_exists="replace")

# Loop through CSV files and create tables
for csv_file in csv_files:
    table_name = csv_file.split("/")[-1].split(".")[0]
    df = pd.read_csv(csv_file)
    create_table(table_name, df)

# Commit changes and close the connection
conn.commit()
conn.close()

print(f"SQLite database '{database_file}' has been created with tables.")


SQLite database 'Trials/parsed_data/my_database.db' has been created with tables.


In [3]:

database_file = "Trials/parsed_data/my_database.db"

conn = sqlite3.connect(database_file)
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS merged;")

create_table_query = '''
CREATE TABLE merged AS
SELECT localization.*, darkness_bins.*, luminecance_annotations.*, protein_sequences.*
FROM localization
LEFT JOIN darkness_bins ON localization.UniprotID = darkness_bins.UniprotID
LEFT JOIN luminecance_annotations ON localization.UniprotID = luminecance_annotations.UniprotID
LEFT JOIN protein_sequences ON localization.UniprotID = protein_sequences.UniprotID;
'''

cursor.execute(create_table_query)

conn.commit()
conn.close()

print("Tables merged and saved as 'merged' in the SQLite database.")


Tables merged and saved as 'merged' in the SQLite database.


In [1]:
import sqlite3
import csv

# Connect to the SQLite database
db_path = 'Trials/parsed_data/my_database.db'  # Replace with your SQLite database file path
connection = sqlite3.connect(db_path)
cursor = connection.cursor()

# Execute a query to select all data from a table (replace 'your_table' with your table name)
query = 'SELECT * FROM merged'
cursor.execute(query)

# Fetch all the rows from the result set
rows = cursor.fetchall()

# Get the column names from the cursor description
columns = [description[0] for description in cursor.description]

# Close the cursor and connection
cursor.close()
connection.close()

# Save the data to a CSV file
csv_path = 'merged.csv'  # Replace with your desired CSV file path
with open(csv_path, 'w', newline='') as csv_file:
    # Create a CSV writer
    csv_writer = csv.writer(csv_file)

    # Write the header (column names) to the CSV file
    csv_writer.writerow(columns)

    # Write the data rows to the CSV file
    csv_writer.writerows(rows)

print(f'Data has been exported to {csv_path}')

Data has been exported to merged.csv


In [ ]:
import csv

input_csv_path = '../merged.csv'    # Replace with the path to your input CSV file
output_csv_path = '../output.csv'  # Replace with the desired output CSV file path

# Specify the new header for the first column
new_first_column_header = 'Uniprot_ID'

with open(input_csv_path, 'r') as input_csv_file:
    csv_reader = csv.reader(input_csv_file)
    
 
    headers = next(csv_reader)
    
 
    if not headers:
        raise ValueError("CSV file has no headers.")
    
 
    headers[0] = new_first_column_header
    
 
    data = list(csv_reader)

with open(output_csv_path, 'w', newline='') as output_csv_file:
    csv_writer = csv.writer(output_csv_file)
    
    # Write the modified headers
    csv_writer.writerow(headers)
    
    # Write the data
    csv_writer.writerows(data)

print(f'The CSV file has been updated with a new header for the first column and saved as {output_csv_path}')
